In [11]:
!pip install -q noisereduce pyloudnorm soundfile librosa transformers torch

import os, json, hashlib, time, random, warnings, shutil, pickle
from pathlib import Path
from typing import Optional, List, Dict, Tuple
from collections import defaultdict

import numpy as np
import librosa
import soundfile as sf
from scipy.signal import fftconvolve
import noisereduce as nr
import torch
from transformers import ClapModel, ClapProcessor
import pyloudnorm as pyln

warnings.filterwarnings('ignore')
random.seed(42)
np.random.seed(42)

print("🟢 [Setup] Libraries imported and deterministic seeds set.")

🟢 [Setup] Libraries imported and deterministic seeds set.


### Wipe Kaggle's Output Storage

In [12]:
import os
import shutil

working_dir = '/kaggle/working'

print("🟡 Wiping Kaggle working directory...")

for item in os.listdir(working_dir):
    item_path = os.path.join(working_dir, item)
    try:
        if os.path.isfile(item_path) or os.path.islink(item_path):
            os.unlink(item_path)
        elif os.path.isdir(item_path):
            shutil.rmtree(item_path)
    except Exception as e:
        print(f"🔴 Failed to delete {item_path}. Reason: {e}")

print("🟢 Kaggle working directory is completely clean.")

🟡 Wiping Kaggle working directory...
🟢 Kaggle working directory is completely clean.


### Remove a Specific Folder/File

In [19]:
import shutil
from pathlib import Path

def delete_target(target_path):
    target = Path(target_path)
    
    if not target.exists():
        print(f"🟡 Target does not exist: {target}")
        return
        
    try:
        if target.is_file() or target.is_symlink():
            target.unlink()
            print(f"🟢 Successfully deleted file: {target}")
        elif target.is_dir():
            shutil.rmtree(target)
            print(f"🟢 Successfully deleted directory: {target}")
    except Exception as e:
        print(f"🔴 Failed to delete {target}. Reason: {e}")

# Just drop your path here
delete_target('/kaggle/working/sterilized_batches')

🟢 Successfully deleted directory: /kaggle/working/sterilized_batches


In [14]:
# ═══════════════════════════════════════════════════════════════════════
# PHASE 0 — Environment & Path Configuration
# ═══════════════════════════════════════════════════════════════════════

# KAGGLE INPUT PATHS
PATHS = {
    'bad_irs':   Path('/kaggle/input/datasets/itorousa/impulse-responses'),
    'mit_irs':   Path('/kaggle/input/datasets/kynthesis/mit-reverb-dataset/MIT_Reverb_Dataset/MIT_Reverb_Dataset'),
    'ljspeech':  Path('/kaggle/input/datasets/dromosys/ljspeech/'),
    'vctk':      Path('/kaggle/input/datasets/kynthesis/vctk-corpus/VCTK-Corpus/wav48'),
#   'langid_en': Path('/kaggle/input/datasets/shrivatssudhir/language-identifier/english/clips')
}

# ⚠️ UPDATE THIS PATH WHEN CHAINING RUNS ⚠️
PREV_RUN_PATH = Path('/kaggle/input/notebooks/itorousa/genesis-data-run1')

# OUTPUT PATHS
OUTPUT          = Path('/kaggle/working')
BATCH_DIR       = OUTPUT / 'batches'
CLAP_DIR        = OUTPUT / 'clap_model'
STERILIZED_DIR  = OUTPUT / 'sterilized_batches'
MIT_IR_DIR      = OUTPUT / 'irs' / 'mit_irs'

for d in [BATCH_DIR, CLAP_DIR, STERILIZED_DIR, MIT_IR_DIR]:
    d.mkdir(parents=True, exist_ok=True)

# AUDIO PARAMETERS & BUDGET
SR            = 48_000
CLIP_SEC      = 5.0
CLIP_SAMPLES  = int(SR * CLIP_SEC)
TRIPLES_PER_BATCH  = 500
MAX_OUTPUT_GB      = 19.0 

print(f"🟢 [Config] Output budget: {MAX_OUTPUT_GB} GB | SR: {SR} | Batch Size: {TRIPLES_PER_BATCH}")

🟢 [Config] Output budget: 19.0 GB | SR: 48000 | Batch Size: 500


In [15]:
# ═══════════════════════════════════════════════════════════════════════
# PHASE 1 — Aggressive Checkpoint Initialization
# ═══════════════════════════════════════════════════════════════════════
CHECKPOINT_PATH = OUTPUT / 'checkpoint.json'

def get_output_size_gb() -> float:
    '''Calculate the exact size of /kaggle/working in GB.'''
    total = sum(f.stat().st_size for f in OUTPUT.rglob('*') if f.is_file())
    return total / (1024 ** 3)

def load_checkpoint() -> dict:
    # 1) Check previous run first (chaining)
    prev_ckpt = PREV_RUN_PATH / 'checkpoint.json'
    if prev_ckpt.exists():
        with open(prev_ckpt) as f:
            ckpt = json.load(f)
        ckpt['run_number'] += 1
        print(f"🟢 [Checkpoint] ♻ Resuming from previous run: {ckpt['triples_completed']} triples done")
        return ckpt

    # 2) Check current working dir (kernel restart mid-session)
    if CHECKPOINT_PATH.exists():
        with open(CHECKPOINT_PATH) as f:
            return json.load(f)

    # 3) Fresh start
    return {
        'batch_id': 0,
        'triples_completed': 0,
        'vocal_cursor': 0,
        'run_number': 1,
    }

def save_checkpoint(ckpt: dict):
    with open(CHECKPOINT_PATH, 'w') as f:
        json.dump(ckpt, f, indent=2)

ckpt = load_checkpoint()
save_checkpoint(ckpt) # Initialize immediately

print(f"🟢 [Checkpoint] Run #{ckpt['run_number']} | Starting at vocal cursor {ckpt['vocal_cursor']}")

🟢 [Checkpoint] ♻ Resuming from previous run: 0 triples done
🟢 [Checkpoint] Run #2 | Starting at vocal cursor 0


In [16]:
# ═══════════════════════════════════════════════════════════════════════
# PHASE 2 — Impulse Response Acquisition & Pooling
# ═══════════════════════════════════════════════════════════════════════
CATALOGUE_PATH = OUTPUT / 'ir_catalogue.json'
prev_catalogue = PREV_RUN_PATH / 'ir_catalogue.json'

bad_pool: List[str] = []
target_pool: List[str] = []
ir_catalogue: Dict = {}

# ─── Fast path: Copy from previous run if exists ───
if prev_catalogue.exists() and not CATALOGUE_PATH.exists():
    shutil.copy2(prev_catalogue, CATALOGUE_PATH)
    print("🟢 [IR Phase] Copied catalogue from previous run.")

if CATALOGUE_PATH.exists():
    print("🟢 [IR Phase] Found existing catalogue. Bypassing extraction.")
    with open(CATALOGUE_PATH, 'r') as f:
        ir_catalogue = json.load(f)
        
    for ir_id, feats in ir_catalogue.items():
        if feats['source'] == 'mit':
            target_pool.append(ir_id)
        else:
            bad_pool.append(ir_id)
            
    print(f"🟢 [IR Phase] Re-hydrated pools -> Target (MIT): {len(target_pool)} | Bad (Custom): {len(bad_pool)}")

else:
    print("🟢 [IR Phase] Processing IRs from scratch...")
    
    # Copy MIT IRs to working directory for easier access later
    if PATHS['mit_irs'].exists():
        shutil.copytree(PATHS['mit_irs'], MIT_IR_DIR, dirs_exist_ok=True)
        
    def process_ir(filepath: Path, source_tag: str):
        try:
            audio, _ = librosa.load(str(filepath), sr=SR, mono=True)
            if len(audio) < 64: return
            peak = np.max(np.abs(audio))
            if peak > 1e-6: audio = audio / peak
            
            ir_id = f"{source_tag}_{filepath.stem}"
            ir_catalogue[ir_id] = {'source': source_tag, 'path': str(filepath)}
            
            if source_tag == 'mit': target_pool.append(ir_id)
            else: bad_pool.append(ir_id)
        except Exception:
            pass

    # Process MIT (Target)
    for f in MIT_IR_DIR.rglob('*.wav'):
        process_ir(f, 'mit')
        
    # Process Custom (Bad)
    for ext in ('*.irs', '*.wav'):
        for f in PATHS['bad_irs'].rglob(ext):
            process_ir(f, 'bad')

    with open(CATALOGUE_PATH, 'w') as f:
        json.dump(ir_catalogue, f, indent=2)
        
    print(f"🟢 [IR Phase] Catalogue built -> Target: {len(target_pool)} | Bad: {len(bad_pool)}")

🟢 [IR Phase] Copied catalogue from previous run.
🟢 [IR Phase] Found existing catalogue. Bypassing extraction.
🟢 [IR Phase] Re-hydrated pools -> Target (MIT): 270 | Bad (Custom): 389


In [17]:
# ═══════════════════════════════════════════════════════════════════════
# PHASE 3 — CLAP Target Embedding Cache
# ═══════════════════════════════════════════════════════════════════════
CLAP_CACHE_PATH = OUTPUT / 'clap_cache.npz'
prev_cache = PREV_RUN_PATH / 'clap_cache.npz'
clap_cache_data = {}

# ─── Fast path: Load from cache ───
if prev_cache.exists() and not CLAP_CACHE_PATH.exists():
    shutil.copy2(prev_cache, CLAP_CACHE_PATH)
    print("🟢 [CLAP Phase] Copied embedding cache from previous run.")

if CLAP_CACHE_PATH.exists():
    print("🟢 [CLAP Phase] Found existing clap_cache.npz. Bypassing model loading.")
    clap_cache_data = dict(np.load(CLAP_CACHE_PATH))
    print(f"🟢 [CLAP Phase] Loaded {len(clap_cache_data)} embeddings.")
else:
    print("🟢 [CLAP Phase] Loading model to compute embeddings...")
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    CLAP_MODEL_ID = "laion/larger_clap_music_and_speech"
    
    # Handle frozen model transfer to avoid re-downloading
    prev_clap_model = PREV_RUN_PATH / 'clap_model'
    if prev_clap_model.exists() and not CLAP_DIR.exists():
        shutil.copytree(prev_clap_model, CLAP_DIR, dirs_exist_ok=True)
        
    if (CLAP_DIR / 'config.json').exists():
        clap_processor = ClapProcessor.from_pretrained(CLAP_DIR)
        clap_model = ClapModel.from_pretrained(CLAP_DIR).to(device).eval()
    else:
        clap_processor = ClapProcessor.from_pretrained(CLAP_MODEL_ID)
        clap_model = ClapModel.from_pretrained(CLAP_MODEL_ID).to(device).eval()
        clap_model.save_pretrained(CLAP_DIR)
        clap_processor.save_pretrained(CLAP_DIR)

    ref_noise = np.random.randn(SR * 3).astype(np.float32) * 0.1
    
    for i, ir_id in enumerate(target_pool):
        if ir_id not in ir_catalogue: continue
        
        ir_audio, _ = librosa.load(ir_catalogue[ir_id]['path'], sr=SR, mono=True)
        scene = fftconvolve(ref_noise, ir_audio, mode='full')[:SR * 3]
        scene = scene / (np.max(np.abs(scene)) + 1e-8)
        
        inputs = clap_processor(audio=scene, sampling_rate=SR, return_tensors="pt")
        inputs = {k: v.to(device) for k, v in inputs.items()}
        
        with torch.no_grad():
            outputs = clap_model.get_audio_features(**inputs)
            # Critical: Must extract pooler_output
            emb = outputs.pooler_output.cpu().numpy().flatten().astype(np.float32)
            
        clap_cache_data[ir_id] = emb
        
    np.savez(CLAP_CACHE_PATH, **clap_cache_data)
    print(f"🟢 [CLAP Phase] Computed and cached {len(clap_cache_data)} embeddings.")

🟢 [CLAP Phase] Copied embedding cache from previous run.
🟢 [CLAP Phase] Found existing clap_cache.npz. Bypassing model loading.
🟢 [CLAP Phase] Loaded 270 embeddings.


In [18]:
# ═══════════════════════════════════════════════════════════════════════
# PHASE 4 — Vocal Sterilization & "Dead" Audio Guarantee
# ═══════════════════════════════════════════════════════════════════════
STATE_FILE = STERILIZED_DIR / 'sterilize_state.json'
prev_sterilized = PREV_RUN_PATH / 'sterilized_batches'

# ─── Fast path: Transfer previous chunks ───
if prev_sterilized.exists() and (prev_sterilized / 'sterilize_state.json').exists():
    if not STATE_FILE.exists():
        shutil.copytree(prev_sterilized, STERILIZED_DIR, dirs_exist_ok=True)
        print("🟢 [Sterilize Phase] Copied sterilized batches from previous run.")

_st = {}
if STATE_FILE.exists():
    with open(STATE_FILE) as f:
        _st = json.load(f)

if _st.get('completed', False):
    print("🟢 [Sterilize Phase] Sterilization previously completed. Bypassing extraction.")
else:
    print("🟢 [Sterilize Phase] Discovering source audio...")
    meter = pyln.Meter(SR)
    all_files = []
    
    for p in PATHS.values():
        if p.name in ['impulse-responses', 'MIT_Reverb_Dataset']: continue
        for ext in ('*.wav', '*.mp3', '*.ogg'):
            all_files.extend([(f, p.name) for f in p.rglob(ext)])
            
    all_files.sort(key=lambda x: str(x[0]))
    random.Random(42).shuffle(all_files)
    print(f"🟢 [Sterilize Phase] Found {len(all_files)} raw vocal files.")
    
    cursor = _st.get('cursor', 0)
    print(f"🟢 [Sterilize Phase] Resuming extraction at file index: {cursor}")
    
    vocal_segments = []
    STERILIZE_CHUNK = 500
    
    for i in range(cursor, len(all_files)):
        if get_output_size_gb() > MAX_OUTPUT_GB:
            print(f"⚠️ Output limit reached. Pausing sterilization.")
            break
            
        fpath, tag = all_files[i]
        try:
            audio, _ = librosa.load(str(fpath), sr=SR, mono=True)
            if len(audio) < SR * 1.5: continue
            
            audio = nr.reduce_noise(y=audio, sr=SR, stationary=True, prop_decrease=0.85)
            audio, _ = librosa.effects.trim(audio, top_db=40)
            if len(audio) < SR * 1.5: continue
            
            loudness = meter.integrated_loudness(audio)
            if loudness > -70: audio = pyln.normalize.loudness(audio, loudness, -23.0)
            
            for start in range(0, len(audio) - SR, CLIP_SAMPLES):
                chunk = audio[start : start + CLIP_SAMPLES]
                if len(chunk) < CLIP_SAMPLES:
                    chunk = np.pad(chunk, (0, CLIP_SAMPLES - len(chunk)))
                
                if np.sqrt(np.mean(chunk ** 2)) < 1e-4: continue
                vocal_segments.append({'audio': chunk.astype(np.float32), 'file': fpath.name, 'dataset': tag})
                
        except Exception: pass
        
        # Flush to disk to protect RAM
        if (i + 1) % STERILIZE_CHUNK == 0 or (i + 1) == len(all_files):
            batch_idx = (i + 1) // STERILIZE_CHUNK
            batch_path = STERILIZED_DIR / f"sterilized_batch_{batch_idx:04d}.pkl"
            with open(batch_path, 'wb') as f:
                pickle.dump(vocal_segments, f)
                
            completed = (i + 1) >= len(all_files)
            with open(STATE_FILE, 'w') as f:
                json.dump({'cursor': i + 1, 'completed': completed}, f)
                
            print(f"  💾 Saved {len(vocal_segments)} segments to {batch_path.name}. RAM cleared.")
            vocal_segments.clear()

# Verification check
total_segs = sum(len(pickle.load(open(f, 'rb'))) for f in STERILIZED_DIR.glob('*.pkl'))
print(f"🟢 [Sterilize Phase] Total sterilized 5.0s segments on disk: {total_segs}")

KeyboardInterrupt: 

---
> ⚠️ **Phase Skip**: Phases 2–4 above can be skipped entirely if a previous run
> completed them successfully. The notebook detects existing `ir_catalogue.json`,
> `clap_cache.npz`, and `sterilize_state.json` to bypass redundant computation.
> Previous run files are located at `/kaggle/input/notebooks/itorousa/genesis-data-run#`.
---


In [26]:
# ═══════════════════════════════════════════════════════════════════════
# PHASE 5 — The Messy DSP Toolkit
# ═══════════════════════════════════════════════════════════════════════
# Random, destructive audio effects applied ONLY to the source input
# to simulate terrible recording environments.

def add_noise(audio: np.ndarray, noise_type: str, snr_db: float) -> np.ndarray:
    n = len(audio)
    if noise_type == 'white': noise = np.random.randn(n)
    elif noise_type == 'pink':
        freqs = np.fft.rfftfreq(n, 1 / SR); freqs[0] = 1
        noise = np.fft.irfft(1.0 / np.sqrt(freqs) * np.exp(2j * np.pi * np.random.rand(len(freqs))))[:n]
    elif noise_type == 'brown':
        noise = np.cumsum(np.random.randn(n)); noise -= np.mean(noise)
    elif noise_type == 'hvac':
        from scipy.signal import butter, sosfilt
        sos = butter(4, [100, 1000], btype='band', fs=SR, output='sos')
        noise = sosfilt(sos, np.random.randn(n))
    else: noise = np.random.randn(n)

    sig_power = np.mean(audio ** 2) + 1e-12
    noise_power = np.mean(noise ** 2) + 1e-12
    noise = noise * np.sqrt((sig_power / (10 ** (snr_db / 10))) / noise_power)
    return audio + noise.astype(np.float32)

def apply_eq(audio: np.ndarray) -> np.ndarray:
    from scipy.signal import butter, sosfilt
    for lo, hi in [(80, 300), (300, 3000), (3000, 12000)]:
        gain_db = random.uniform(-6, 6)
        if abs(gain_db) >= 1:
            try:
                sos = butter(2, [lo, hi], btype='band', fs=SR, output='sos')
                audio += sosfilt(sos, audio) * ((10 ** (gain_db / 20)) - 1)
            except Exception: pass
    return audio.astype(np.float32)

def apply_highpass(audio: np.ndarray) -> np.ndarray:
    from scipy.signal import butter, sosfilt
    sos = butter(4, random.uniform(60, 300), btype='high', fs=SR, output='sos')
    return sosfilt(sos, audio).astype(np.float32)

def apply_lowpass(audio: np.ndarray) -> np.ndarray:
    from scipy.signal import butter, sosfilt
    sos = butter(4, random.uniform(3000, 16000), btype='low', fs=SR, output='sos')
    return sosfilt(sos, audio).astype(np.float32)

def apply_bitcrush(audio: np.ndarray) -> np.ndarray:
    levels = 2 ** random.randint(8, 16)
    return (np.round(audio * levels) / levels).astype(np.float32)

def apply_hard_clip(audio: np.ndarray) -> np.ndarray:
    threshold = random.uniform(0.3, 0.9)
    return np.clip(audio, -threshold, threshold).astype(np.float32)

DEGRADATIONS = {
    'noise':    lambda a: add_noise(a, random.choice(['white', 'pink', 'brown', 'hvac']), random.uniform(5, 40)),
    'eq':       apply_eq,
    'highpass': apply_highpass,
    'lowpass':  apply_lowpass,
    'gain':     lambda a: a * (10 ** (random.uniform(-6, 6) / 20)),
    'bitcrush': apply_bitcrush,
    'clip':     apply_hard_clip,
}

def apply_random_degradations(audio: np.ndarray, min_augs=3, max_augs=6) -> Tuple[np.ndarray, List[str]]:
    chosen = random.sample(list(DEGRADATIONS.keys()), random.randint(min_augs, max_augs))
    for name in chosen: audio = DEGRADATIONS[name](audio)
    return np.clip(audio, -1.0, 1.0).astype(np.float32), chosen

print("🟢 [Degradation] Toolkit loaded.")

🟢 [Degradation] Toolkit loaded.


In [28]:
# ═══════════════════════════════════════════════════════════════════════
# PHASE 6 — The Streaming Training Data Engine
# ═══════════════════════════════════════════════════════════════════════
def convolve_trim(vocal: np.ndarray, ir: np.ndarray) -> np.ndarray:
    wet = fftconvolve(vocal, ir, mode='full')[:CLIP_SAMPLES]
    peak = np.max(np.abs(wet))
    return (wet / peak).astype(np.float32) if peak > 1e-6 else wet.astype(np.float32)

def audio_to_int16(audio: np.ndarray) -> np.ndarray:
    return (np.clip(audio, -1, 1) * 32767).astype(np.int16)

# ── Dynamic IR Audio Loading (Path-Drift Proof) ──
loaded_irs = {}
print("🟢 [Engine] Mapping available IR audio files across all directories...")

# 1. Build a lookup table of ALL .wav and .irs files on the machine
audio_lookup = {}
for search_dir in [MIT_IR_DIR, PATHS['bad_irs'], PREV_RUN_PATH / 'irs']:
    if search_dir.exists():
        for f in search_dir.rglob('*'):
            if f.suffix.lower() in ['.wav', '.irs'] and f.is_file():
                audio_lookup[f.name] = f
                
# 2. Load the IRs using the lookup table
if not CATALOGUE_PATH.exists():
    print("🔴 [Engine] FATAL: ir_catalogue.json is missing!")
else:
    with open(CATALOGUE_PATH, 'r') as f: ir_cat = json.load(f)
    for ir_id, data in ir_cat.items():
        original_name = Path(data['path']).name
        if original_name in audio_lookup:
            try:
                a, _ = librosa.load(str(audio_lookup[original_name]), sr=SR, mono=True)
                peak = np.max(np.abs(a))
                loaded_irs[ir_id] = (a / peak).astype(np.float32) if peak > 1e-6 else a.astype(np.float32)
            except Exception: pass

print(f"🟢 [Engine] Successfully loaded {len(loaded_irs)} IRs into RAM.")
if len(loaded_irs) == 0:
    print("🔴 [Engine] FATAL: Could not load any IR audio! Generation loop will skip all segments.")

# ── Robust PKL Discovery ──
pkl_files = []
if (PREV_RUN_PATH / 'sterilized_batches').exists():
    pkl_files.extend(list((PREV_RUN_PATH / 'sterilized_batches').glob('*.pkl')))
pkl_files.extend(list(STERILIZED_DIR.glob('*.pkl')))

unique_pkls = {p.name: p for p in pkl_files}
pkl_files = sorted(unique_pkls.values(), key=lambda x: x.name)

print(f"🟢 [Engine] Found {len(pkl_files)} sterilized batch files.")
print(f"🟢 [Engine] Pool Sizes -> Target: {len(target_pool)} | Bad: {len(bad_pool)}")

# ── Setup Variables ──
clap_cache_data = dict(np.load(CLAP_CACHE_PATH)) if CLAP_CACHE_PATH.exists() else {}
batch_id = ckpt.get('batch_id', 0)
total_completed = ckpt.get('triples_completed', 0)

scenarios = ['1', '1a', '1b', '1c', '2', '3', '3a', '4', '5']
weights =   [ 0.07, 0.04, 0.03, 0.06, 0.20, 0.07, 0.13, 0.20, 0.20] 

batch_sources, batch_targets, batch_claps, batch_meta = [], [], [], []

print(f"\n🟢 [Engine] Starting streaming triple generation. Checkpoint: {total_completed} completed.")
t_start = time.time()
limit_reached = False

for pkl_file in pkl_files:
    if limit_reached: break
    print(f"\n  → Processing {pkl_file.name}...")
    
    with open(pkl_file, 'rb') as f: chunk_segments = pickle.load(f)
    random.shuffle(chunk_segments)
    
    skipped_ir = 0
    skipped_qa = 0
    
    for seg in chunk_segments:
        if get_output_size_gb() > MAX_OUTPUT_GB:
            print(f"\n⚠️ Output limit reached ({MAX_OUTPUT_GB} GB). Stopping generation.")
            limit_reached = True
            break
            
        V = seg['audio']
        scen = random.choices(scenarios, weights=weights, k=1)[0]
        bad_ir = random.choice(bad_pool) if bad_pool else None
        tgt_ir = random.choice(target_pool) if target_pool else None
        tgt_ir2 = random.choice(target_pool) if target_pool else None
        
        # Guardrail: If IR audio is missing from RAM, skip and tally
        if not bad_ir or not tgt_ir or bad_ir not in loaded_irs or tgt_ir not in loaded_irs: 
            skipped_ir += 1
            continue

        source_audio = V.copy()
        target_audio = V.copy()
        used_target_ir = tgt_ir
        
        # Execute the 20k Scenario Matrix Mathematics
        if scen == '1': 
            target_audio = convolve_trim(V, loaded_irs[tgt_ir])
            source_audio = add_noise(target_audio, 'white', random.uniform(5, 20))
        elif scen == '1a': 
            source_audio = convolve_trim(V, loaded_irs[tgt_ir])
            source_audio = add_noise(source_audio, 'white', random.uniform(5, 20))
        elif scen == '1b': 
            target_audio = convolve_trim(V, loaded_irs[tgt_ir])
            source_audio = add_noise(V, 'white', random.uniform(5, 20))
        elif scen == '1c': 
            target_audio = convolve_trim(V, loaded_irs[tgt_ir])
            source_audio = convolve_trim(V, loaded_irs[tgt_ir2])
            source_audio = add_noise(source_audio, 'white', random.uniform(5, 20))
        elif scen == '2': 
            target_audio = convolve_trim(V, loaded_irs[tgt_ir])
            source_audio = apply_lowpass(apply_highpass(target_audio))
        elif scen == '3': 
            source_audio = convolve_trim(V, loaded_irs[bad_ir])
        elif scen == '3a': 
            target_audio = convolve_trim(V, loaded_irs[tgt_ir])
            source_audio = convolve_trim(V, loaded_irs[bad_ir])
        elif scen == '4': 
            target_audio = convolve_trim(V, loaded_irs[tgt_ir])
        elif scen == '5': 
            source_audio, _ = apply_random_degradations(V)

        # QA Checks: Skip completely silent or mathematically corrupted tensors
        if np.sqrt(np.mean(source_audio**2)) < 1e-4 or np.sqrt(np.mean(target_audio**2)) < 1e-4: 
            skipped_qa += 1
            continue
        if np.any(np.isnan(source_audio)) or np.any(np.isnan(target_audio)): 
            skipped_qa += 1
            continue

        # CLAP lookup
        tgt_clap = clap_cache_data.get(used_target_ir) if scen not in ['1a', '3', '5'] else np.zeros(512, dtype=np.float32)
        if tgt_clap is None: tgt_clap = np.zeros(512, dtype=np.float32)

        batch_sources.append(audio_to_int16(source_audio))
        batch_targets.append(audio_to_int16(target_audio))
        batch_claps.append(tgt_clap)
        batch_meta.append({'scenario': scen, 'vocal': seg['file'], 'bad_ir': bad_ir, 'tgt_ir': tgt_ir, 'tgt_ir2': tgt_ir2 if scen == '1c' else None})

        # Save batch when full
        if len(batch_sources) >= TRIPLES_PER_BATCH:
            np.savez(BATCH_DIR / f'batch_{batch_id:04d}.npz', source_audio=np.stack(batch_sources), target_audio=np.stack(batch_targets), target_clap=np.stack(batch_claps))
            with open(BATCH_DIR / f'batch_{batch_id:04d}_meta.json', 'w') as f: json.dump(batch_meta, f)
            
            total_completed += len(batch_sources)
            batch_id += 1
            ckpt.update({'batch_id': batch_id, 'triples_completed': total_completed})
            save_checkpoint(ckpt)
            
            rate = len(batch_sources) / (time.time() - t_start)
            print(f"    💾 Batch {batch_id-1:04d} saved | {total_completed:,} total | {get_output_size_gb():.1f} GB")
            
            batch_sources.clear(); batch_targets.clear(); batch_claps.clear(); batch_meta.clear()
            t_start = time.time()
            
    # Print metrics for the chunk
    buffered = len(batch_sources)
    print(f"    ↳ Chunk Complete | Buffered: {buffered} | Skipped (Missing IR): {skipped_ir} | Skipped (QA): {skipped_qa}")
    del chunk_segments # Protect RAM

🟢 [Engine] Mapping available IR audio files across all directories...
🟢 [Engine] Successfully loaded 659 IRs into RAM.
🟢 [Engine] Found 34 sterilized batch files.
🟢 [Engine] Pool Sizes -> Target: 270 | Bad: 389

🟢 [Engine] Starting streaming triple generation. Checkpoint: 0 completed.

  → Processing sterilized_batch_0001.pkl...
    💾 Batch 0000 saved | 500 total | 0.4 GB
    ↳ Chunk Complete | Buffered: 64 | Skipped (Missing IR): 0 | Skipped (QA): 0

  → Processing sterilized_batch_0003.pkl...
    💾 Batch 0001 saved | 1,000 total | 0.9 GB
    💾 Batch 0002 saved | 1,500 total | 1.3 GB
    ↳ Chunk Complete | Buffered: 167 | Skipped (Missing IR): 0 | Skipped (QA): 0

  → Processing sterilized_batch_0004.pkl...
    💾 Batch 0003 saved | 2,000 total | 1.8 GB
    ↳ Chunk Complete | Buffered: 202 | Skipped (Missing IR): 0 | Skipped (QA): 0

  → Processing sterilized_batch_0005.pkl...
    💾 Batch 0004 saved | 2,500 total | 2.2 GB
    ↳ Chunk Complete | Buffered: 233 | Skipped (Missing IR): 0 |

---
> ⚠️ **Phase Skip**: Phases 5–6 above can be skipped if a previous run already
> generated sufficient training batches. Attach the previous output as input and
> the checkpoint system will resume from where it left off.
---


In [29]:
# ═══════════════════════════════════════════════════════════════════════
# PHASE 7 — Manifest & Pipeline Conclusion
# ═══════════════════════════════════════════════════════════════════════

# ── Flush remaining triples ──
if batch_sources:
    np.savez(BATCH_DIR / f'batch_{batch_id:04d}.npz', source_audio=np.stack(batch_sources), target_audio=np.stack(batch_targets), target_clap=np.stack(batch_claps))
    with open(BATCH_DIR / f'batch_{batch_id:04d}_meta.json', 'w') as f: json.dump(batch_meta, f)
    total_completed += len(batch_sources)
    batch_id += 1
    ckpt.update({'batch_id': batch_id, 'triples_completed': total_completed})
    save_checkpoint(ckpt)

# ── Compute Checksums ──
def sha256_file(path: Path) -> str:
    h = hashlib.sha256()
    with open(path, 'rb') as f:
        for chunk in iter(lambda: f.read(8192), b''): h.update(chunk)
    return h.hexdigest()

print("🟢 [Manifest] Computing SHA-256 checksums...")
manifest = {
    'run_number': ckpt['run_number'],
    'triples_total': ckpt['triples_completed'],
    'batches': ckpt['batch_id'],
    'sample_rate': SR, 'clip_samples': CLIP_SAMPLES,
    'output_size_gb': round(get_output_size_gb(), 3),
    'batch_checksums': {f.name: sha256_file(f) for f in sorted(BATCH_DIR.glob('batch_*.npz'))}
}

with open(OUTPUT / 'manifest.json', 'w') as f: json.dump(manifest, f, indent=2)

print(f"\n{'='*60}")
print(f"  GENESIS DATA CURATION — RUN {ckpt['run_number']} COMPLETE")
print(f"{'='*60}")
print(f"  Total triples:    {manifest['triples_total']:,}")
print(f"  Batches output:   {manifest['batches']}")
print(f"  Output size:      {manifest['output_size_gb']:.2f} GB")
print(f"{'='*60}")

if limit_reached:
    print(f"\n⚠️  BUDGET LIMIT REACHED. To continue:")
    print(f"    1. Save this notebook's output as a Kaggle dataset.")
    print(f"    2. Update PREV_RUN_PATH in Cell 2 to point to it.")
    print(f"    3. Restart and run all. Checkpoints will automatically resume.")
else:
    print(f"\n🟢 All sterilized vocals processed. Dataset complete!")

🟢 [Manifest] Computing SHA-256 checksums...

  GENESIS DATA CURATION — RUN 2 COMPLETE
  Total triples:    20,623
  Batches output:   42
  Output size:      18.48 GB

🟢 All sterilized vocals processed. Dataset complete!


In [30]:
from pathlib import Path

def print_tree(dir_path="/kaggle/working"):
    base = Path(dir_path)
    
    if not base.exists():
        print(f"🔴 Directory not found: {dir_path}")
        return
        
    print(f"🟢 {base.resolve()}")
    
    # Recursively grab everything and sort it
    for path in sorted(base.rglob('*')):
        depth = len(path.relative_to(base).parts)
        # Create the branching indentation
        indent = '│   ' * (depth - 1) + '├── '
        print(f"{indent}{path.name}")

# Run it
print_tree()

🟢 /kaggle/working
├── .virtual_documents
│   ├── __notebook_source__.ipynb
├── batches
│   ├── batch_0000.npz
│   ├── batch_0000_meta.json
│   ├── batch_0001.npz
│   ├── batch_0001_meta.json
│   ├── batch_0002.npz
│   ├── batch_0002_meta.json
│   ├── batch_0003.npz
│   ├── batch_0003_meta.json
│   ├── batch_0004.npz
│   ├── batch_0004_meta.json
│   ├── batch_0005.npz
│   ├── batch_0005_meta.json
│   ├── batch_0006.npz
│   ├── batch_0006_meta.json
│   ├── batch_0007.npz
│   ├── batch_0007_meta.json
│   ├── batch_0008.npz
│   ├── batch_0008_meta.json
│   ├── batch_0009.npz
│   ├── batch_0009_meta.json
│   ├── batch_0010.npz
│   ├── batch_0010_meta.json
│   ├── batch_0011.npz
│   ├── batch_0011_meta.json
│   ├── batch_0012.npz
│   ├── batch_0012_meta.json
│   ├── batch_0013.npz
│   ├── batch_0013_meta.json
│   ├── batch_0014.npz
│   ├── batch_0014_meta.json
│   ├── batch_0015.npz
│   ├── batch_0015_meta.json
│   ├── batch_0016.npz
│   ├── batch_0016_meta.json
│   ├── batch_0017.npz
│   ├─

## 🔗 Checkpoint Chaining (20 GB Limit)

If the output hit the size limit before processing all vocals:

1. **Save this notebook's output** as a Kaggle dataset (e.g. `genesis-data-run1`)
2. **Create a new notebook** (or re-run this one) and attach:
   - All the same input datasets (IRs, LJSpeech, VCTK, Language Identifier)
   - The previous output as input (update `PREV_RUN_PATH` in Cell 3)
3. **Run all cells** — the checkpoint system automatically skips completed work

Each run produces ~19 GB of training triples. Chain as many times as needed.

### Using the Data in Training

```python
# In the training notebook, load all batches from all runs:
import numpy as np
from pathlib import Path

run_dirs = [
    Path('/kaggle/input/genesis-data-run1/batches'),
    Path('/kaggle/input/genesis-data-run2/batches'),
    # ... add more runs
]

for run_dir in run_dirs:
    for batch_file in sorted(run_dir.glob('batch_*.npz')):
        data = np.load(batch_file)
        source_audio = data['source_audio']   # (N, 240000) int16
        target_audio = data['target_audio']   # (N, 240000) int16
        target_clap  = data['target_clap']    # (N, CLAP_DIM) float32
        # Convert int16 back to float32: audio = source_audio.astype(np.float32) / 32767
        # Compute STFT on-the-fly during training for memory efficiency
```
